In [1]:
import os

import torch
import torchaudio
import soxr

# Navigate from src/scripts/ up to project root
project_root = os.path.abspath(os.path.join(os.getcwd(), "..",".."))
os.chdir(project_root)

from src.models.detector import RobustDetector

device = "cpu"

convolve = True  # Set to False to disable convolutional during inference
ckpt_path = os.path.join(project_root, "src/checkpoints/models/robustdetector-stft-use_conv=False.ckpt")
audio_path = os.path.join(project_root, "data/signals/spedup/sunov5_s102.mp3")

model = RobustDetector.load_from_checkpoint(ckpt_path)
model.eval()
model.to(device)

waveform, sr = torchaudio.load(audio_path)
if sr != model.hparams.sampling_rate:
    print(f"Resampling {audio_path} from {sr} Hz to {model.hparams.sampling_rate} Hz")
    waveform = soxr.resample(waveform.T, sr, model.hparams.sampling_rate).T
    waveform = torch.from_numpy(waveform)

waveform = waveform.to(device)

predictions = model.predict(waveform, convolve=convolve)
predictions

0.9713748693466187